# Шаблон для инференса моделей

**Автор:** [Ваше имя]  
**Дата:** [Дата]  
**Модель:** [Название модели]  

## Описание

Этот ноутбук предназначен для тестирования модели распознавания рукописного текста.

**Задачи:**
1. Загрузить предобученную модель
2. Выполнить инференс на тестовых данных
3. Замерить производительность (время, память)
4. Сохранить результаты

## 1. Подключение Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 2. Установка зависимостей

In [ ]:
# Клонирование репозитория
!git clone https://github.com/Nadarsa/handwritten-ocr.git
%cd handwritten-ocr

# Установка зависимостей
!pip install -q -r requirements.txt

## 3. Настройка путей и импорты

In [ ]:
import os
import sys
import time
import json
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm

# Добавить src в Python path
sys.path.append('/content/handwritten-ocr/src')

# Импорт модулей проекта
from models.inference import TrOCRInference, EasyOCRInference, ModelBenchmark

# Пути к данным
PROJECT_PATH = '/content/drive/MyDrive/Практикум_3_семестр/handwritten-ocr'
DATA_PATH = os.path.join(PROJECT_PATH, 'data/processed/hwr200')
RESULTS_PATH = os.path.join(PROJECT_PATH, 'results')

print(f"Путь к данным: {DATA_PATH}")
print(f"Данные существуют: {os.path.exists(DATA_PATH)}")

## 4. Загрузка данных

In [ ]:
# Функция для загрузки изображений
import cv2

def load_test_images(data_path, max_images=50):
    """Загружает тестовые изображения."""
    images_path = Path(data_path) / 'images'
    
    if not images_path.exists():
        print(f"Папка не найдена: {images_path}")
        return [], []
    
    image_files = list(images_path.glob('*.jpg')) + list(images_path.glob('*.png'))
    image_files = sorted(image_files)[:max_images]
    
    images = []
    image_paths = []
    
    for img_path in tqdm(image_files, desc="Загрузка изображений"):
        img = cv2.imread(str(img_path))
        if img is not None:
            images.append(img)
            image_paths.append(str(img_path))
    
    return images, image_paths

# Загрузка
test_images, test_image_paths = load_test_images(DATA_PATH, max_images=50)
print(f"Загружено изображений: {len(test_images)}")

## 5. Загрузка модели

**Замените на вашу модель:**
- TrOCRInference
- EasyOCRInference
- Другая модель

In [ ]:
# Выбор устройства
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Используется устройство: {device}")

# Загрузка модели (ВЫБЕРИТЕ ОДНУ)
# model = TrOCRInference(device=device)
# model = EasyOCRInference(languages=['ru'], device=device)

# Пример с TrOCR
model = TrOCRInference(model_name='microsoft/trocr-base-handwritten', device=device)

## 6. Тестовый инференс на одном изображении

In [ ]:
import matplotlib.pyplot as plt

if test_images:
    # Взять первое изображение
    test_img = test_images[0]
    
    # Инференс
    start_time = time.time()
    prediction = model.predict(test_img)
    inference_time = (time.time() - start_time) * 1000  # в мс
    
    # Визуализация
    plt.figure(figsize=(12, 4))
    plt.imshow(cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title(f'Распознано: "{prediction}" | Время: {inference_time:.2f} мс')
    plt.show()
    
    print(f"Текст: {prediction}")
    print(f"Время инференса: {inference_time:.2f} мс")

## 7. Инференс на всем тестовом наборе

In [ ]:
# Инференс с логированием времени
predictions = []
inference_times = []

for img in tqdm(test_images, desc="Инференс"):
    start_time = time.time()
    pred = model.predict(img)
    inference_time = (time.time() - start_time) * 1000
    
    predictions.append(pred)
    inference_times.append(inference_time)

print(f"\nИнференс завершен!")
print(f"Обработано изображений: {len(predictions)}")
print(f"Среднее время инференса: {np.mean(inference_times):.2f} мс")

## 8. Анализ производительности

In [ ]:
# Статистика по времени
print("Статистика производительности:")
print(f"Среднее время: {np.mean(inference_times):.2f} мс")
print(f"Медиана: {np.median(inference_times):.2f} мс")
print(f"Мин. время: {np.min(inference_times):.2f} мс")
print(f"Макс. время: {np.max(inference_times):.2f} мс")
print(f"Std: {np.std(inference_times):.2f} мс")

# График распределения времени
plt.figure(figsize=(10, 4))
plt.hist(inference_times, bins=30, edgecolor='black')
plt.xlabel('Время инференса (мс)')
plt.ylabel('Частота')
plt.title('Распределение времени инференса')
plt.axvline(np.mean(inference_times), color='red', linestyle='--', label=f'Среднее: {np.mean(inference_times):.2f} мс')
plt.legend()
plt.show()

## 9. Сохранение результатов

In [ ]:
# Имя эксперимента
EXPERIMENT_NAME = 'trocr_hwr200_test'  # ИЗМЕНИТЕ НА ВАШЕ НАЗВАНИЕ

# Создать папку для результатов
experiment_dir = os.path.join(RESULTS_PATH, EXPERIMENT_NAME)
os.makedirs(experiment_dir, exist_ok=True)

# Сохранить предсказания
df = pd.DataFrame({
    'image_path': test_image_paths,
    'predicted_text': predictions,
    'inference_time_ms': inference_times
})

predictions_path = os.path.join(experiment_dir, 'predictions.csv')
df.to_csv(predictions_path, index=False, encoding='utf-8')
print(f"Предсказания сохранены: {predictions_path}")

# Сохранить метрики производительности
performance_metrics = {
    'model_name': model.model_name,
    'device': device,
    'num_samples': len(test_images),
    'avg_inference_time_ms': float(np.mean(inference_times)),
    'median_inference_time_ms': float(np.median(inference_times)),
    'min_inference_time_ms': float(np.min(inference_times)),
    'max_inference_time_ms': float(np.max(inference_times)),
    'std_inference_time_ms': float(np.std(inference_times))
}

metrics_path = os.path.join(experiment_dir, 'performance_metrics.json')
with open(metrics_path, 'w', encoding='utf-8') as f:
    json.dump(performance_metrics, f, ensure_ascii=False, indent=2)

print(f"Метрики сохранены: {metrics_path}")
print("\nГотово! Результаты сохранены на Google Drive.")

## 10. Примеры распознавания

In [ ]:
# Показать несколько примеров
num_examples = min(5, len(test_images))

fig, axes = plt.subplots(num_examples, 1, figsize=(12, 3 * num_examples))
if num_examples == 1:
    axes = [axes]

for i, (img, pred) in enumerate(zip(test_images[:num_examples], predictions[:num_examples])):
    axes[i].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    axes[i].axis('off')
    axes[i].set_title(f'Распознано: "{pred}"', fontsize=12)

plt.tight_layout()
plt.show()

## Выводы

**Напишите ваши выводы:**
- Как работает модель?
- Какая производительность?
- Какие проблемы заметили?
- Рекомендации для улучшения

---

**Следующие шаги:**
1. Вычислить метрики точности (CER, WER) - см. metrics_template.ipynb
2. Сравнить с другими моделями
3. Проанализировать типичные ошибки